## Efficient Protein Ground State Energy Computation via Fragmentation and Reassembly

### Motivation

The objective from a biological standpoint is to address the challenge of efficiently simulating proteins by fragmenting them into amino acids. Proteins are the fundamental basis for all interactions in the body, and their characterization is essential for understanding biological processes and advancing drug discovery. Notably, all proteins in the human body are composed of the same 20 amino acids. Therefore, we aim to develop a method for fragmenting proteins into their corresponding amino acids and then reassembling them post-simulation, incorporating chemical corrections to ensure the accurate reconstitution of the original molecule.

This work is therefore based on a set of hypotheses and assumptions that guide the design and development of the proposed methodology. First, it is assumed that large molecules, such as proteins, can be fragmented into amino acids without significantly compromising the global properties of the molecule. This assumption is supported by the fact that amino acids are chemically well-defined and biologically relevant units that can be studied independently and reassembled. Second, it is further assumed that any residual interactions between fragments can be addressed through specific energetic adjustments. Third and last, it is acknowledged that current quantum computers are noisy and limited in terms of available qubits, so the proposed methodology is designed to ensure the efficient implementation of the strategy.

Here we present with the detailed pipeline on how to compute the GSE for a small peptide through fragmentation, reassembly, and chemical corrections. The last section also presents with an already prepared function to automatically return the energy of your own protein by introducing a few charactestics.

### Pipeline

First of all, we developed a database containing the 20 essential amino acids with some of their main characteristics and their ground state energy computed classically. This enabled us to automated the process of fragmentation and reassembly of each amino acid. The following code gives an idea on how the database was constructed (you have it as csv format in the data folder):

In [1]:
import pandas as pd
data = pd.read_csv("C:/Users/lclai/Desktop/protein_fragmentation/data/aminoacids.csv") # here change the path corresponding to your directory
data

,id,aa,mf,energy,mw,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,exactmass,monoisotopicmass,covalentunitcnt,isotopeatomcnt,totalatomstereocnt,definedatomstereocnt,electrones,S
0,6274.0,hystidine,C6H9N3O2,-538.52442,155.15,92.0,151.0,-3.2,11.0,3.0,4.0,3.0,155.0690,155.0690,1.0,0.0,1.0,1.0,82.0,0.0
1,6306.0,leucine,C6H13NO2,-433.42225,131.17,63.3,103.0,-1.7,9.0,2.0,3.0,3.0,131.0950,131.0950,1.0,0.0,2.0,2.0,72.0,0.0
2,6106.0,isoleucine,C6H13NO2,-432.82708,131.17,63.3,101.0,-1.5,9.0,2.0,3.0,3.0,131.0950,131.0950,1.0,0.0,1.0,1.0,72.0,0.0
3,5962.0,lysine,C6H14N2O2,-487.74060,146.19,89.3,106.0,-3.0,10.0,3.0,4.0,5.0,146.1060,146.1060,1.0,0.0,1.0,1.0,80.0,0.0
4,6137.0,methionine,C5H11NO2S,-788.02138,149.21,88.6,97.0,-1.9,9.0,2.0,4.0,4.0,149.0510,149.0510,1.0,0.0,1.0,1.0,80.0,1.0
5,6140.0,phenylalanine,C9H11NO2,-544.43743,165.19,63.3,153.0,-1.5,12.0,2.0,3.0,3.0,165.0790,165.0790,1.0,0.0,1.0,1.0,88.0,0.0
6,6288.0,threonine,C4H9NO3,-430.09637,119.12,83.6,93.3,-2.9,8.0,3.0,4.0,2.0,119.0580,119.0580,1.0,0.0,2.0,2.0,64.0,0.0
7,6305.0,tryptophan,C11H12N2O2,-673.57377,204.22,79.1,245.0,-1.1,15.0,3.0,3.0,3.0,204.0900,204.0900,1.0,0.0,1.0,1.0,108.0,0.0
8,6287.0,valine,C5H11NO2,-394.84749,117.15,63.3,90.4,-2.3,8.0,2.0,3.0,2.0,117.0790,117.0790,1.0,0.0,1.0,1.0,64.0,0.0
9,6322.0,arginine,C6H14N4O2,-595.17254,174.20,128.0,176.0,-4.2,12.0,4.0,4.0,5.0,174.1120,174.1120,1.0,0.0,1.0,1.0,94.0,0.0


And for every protein we can think of we can obtain the energies from the amino acids present in its sequence. For example, if we were interest in obtaining the energy for a the following protein:

SER - GLY - GLUT

We would have to filter and obtain the energi corresponding to the amino acids serine (SER), glycine (GLY) and glutamic acid (GLU).

In [2]:
energy_glycine = data.loc[data['aa'] == 'glycine', 'energy'].values[0]
energy_serine = data.loc[data['aa'] == 'serine', 'energy'].values[0]
energy_glutamic_acid = data.loc[data['aa'] == 'glutamic acid', 'energy'].values[0]

So, to summarize, the first steo would be to fragment our peptide into its corresponding amino acids. This is a trivial step since all proteins are by definition described as their amino acid sequence.

We can crealy see this partition in the protein that we have just mentioned:

<img src="1.png" width="400"/>

Once the protein has been fragmented, the next steps is to compute the energy corresponding to each of the fragments. However, since we already have the energy of every amino acid in the same dataabse we can jump directly to the next step that would be reassembly.

For that, our strategy consists on implementing the following equation: 

$$
E_{m} = \sum_{i=1}^{n} E_{f_i} \pm \sum_{j=1}^{k} E_{am_j}
$$

Where $n$ is the protein's total number of fragments generated, $E_{f_i}$ is the GSE corresponding to each fragment, $k$ is the total number of molecules lost or added within the process, $E_{am_j}$ corresponds to the GSE of each additional molecule, and $E_m$ corresponds to the final GSE of the molecule obtained through reassembly. Here, $i$ is the index that enumerates the $n$ fragments of the protein generated, with $i = 1, 2, \dots, n$. Similarly, $j$ represents the index of the $k$ additional or removed molecules during the process, with $j = 1, 2, \dots, k$.

To the identify additional molecules one has to analize the original molecule before the fragmentation. For example, tipically if the bond is formed betweent he cabroxi and the amino terminal groups of the amino acids, a molecula of water is going to be released. Another case could be that of aspartame, where we have ASP-PHE-CH3, which means we would have to add the energy corresponding to the methyl group (CH3). 

Now that we know how this is done, let's do it for the SER-GLY-GLU peptide. As mentioned, we first fragtioned this chain into three different amino acids. If we take a look at the previous chain we can see that the three amino acids are bonded through typical peptidic bonds, which mean we would have to account for the release of a total of 2 water molecules. Furthermore, all of them present their physiological conformation, meaning we will not have to account for the energy of any additional group.

In conclusion, this means that the final energy for this specific protein chain will we the sum of each amino acid energy minus two times the energy of a water molecule. This process can be easily understood by taking a look into the following image:

<img src="3.png" width="400"/>

For the additional molecules the energy can be calculated classically (take a look in the code we provide in "ground_truth") or from quantum chemistry databases such as CCCBDB (https://cccbdb.nist.gov/energy1x.asp).

All these operations are implemented in the following code:

In [3]:
energy_water = -74.9659 #GSE of the water molecule obtained form the cccbdb database

total_energy_gly_ser_glu = energy_glycine + energy_serine + energy_glutamic_acid #summing all the energies

reassembly_gly_ser_glu = total_energy_gly_ser_glu- 2*energy_water #adding chemical corrections corresponding to bond formation

print(f"Energy of gly-val-ala through reassembly is {reassembly_gly_ser_glu} Hartrees") #final energy of our peptide

Energy of gly-val-ala through reassembly is -1062.0454499999998 Hartrees


Finally, in case we have the ground truth energy of our protein of interest, we can compute the relative error between our strategy and such energy by implementing the following function:

In [5]:
import numpy as np

gt_gly_val_ala = -1062.00937 #define here manually your ground truth

def relative_error(gt,computed):
    '''function to compute the relative percentage error between the reassembly and the ground
    truth energy, the arguments correspond to these energies, respectively'''
    dif = np.abs(gt-computed)
    error = dif/np.abs(gt)
    percentage_error = error * 100
    return percentage_error
    
relative_error_gly_val_ala = relative_error(gt_gly_val_ala,reassembly_gly_ser_glu)
print(f"The RE% of the reassebly for the gly-val-ala peptide is {relative_error_gly_val_ala} %")

The RE% of the reassebly for the gly-val-ala peptide is 0.0033973334905563245 %


### Automating the process for you

Lastly, if you want to directly obtain the energy of a certain peptide you can just run the following cell and input the information that is going to be asked or otherwise you can leave it blank if you do not know it, but you will be probably facing higher error rates. Use the previous step to step tutorial to understand how the function computes the energies.

Here the following cell is runned inputing the following:

- Protein sequence: gly-ser-glu
- Number of water molecules lost: 2
- Additional molecules: blank
- Ground truth energy: -1062.00937

We can see how we obtain the same results but way faster.

In [1]:
from energy_computation import compute_protein_energy

total_energy = compute_protein_energy()

The final computed energy of ser-gly-glu is: -1062.04545 units
The error of reassembly of ser-gly-glu is of 0.00340%


### Hope this has been useful with your calculations and do not forget to cite us if you use this code!